# Tutorial 4: Path Planning
The previous tutorials focused on localization, one of the key domains of mobile robotics. This week we will be looking at the other key domain of mobile robotics, navigation, which is the problem of guiding a robot towards a specific goal using a map. Map-based navigation is also called motion or path planning. 

# Part A: Graph Search Algorithms

## Import Packages

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

## Outline of Part A
The first part of the tutorial considers Graph Search algorithms, which are a type of algorithm used for path planning. Below are the learning objectives to be achieved by the end of this part of the tutorial.

**Objectives:**
* Implement graph search algorithms for robot motion planning in discretized state-spaces 
* Learn the differences between Dijkstra's and A* algorithms

## A* Algorithm

**Main idea:**
* Encode grid-like (discrete) state space onto a graph
* Find the shortest path from the start node to goal node by traversing the graph

**Description:**
* The implementation of informed-search algorithms like Dijkstra's and A* use a priority queue to keep track and identify the best node from the search frontier to expand from. This priority queue is known as the open set or fringe. At each step of the algorithm, the best node is removed from the queue, the cost values of its neighbors are updated, and these neighbors are added to the queue (search frontier).

**Main difference between Dijkstra's and A* algorithm:**
* A* algorithm uses an additional heuristic to guide its search for a better performance, which is more efficient than the Dijkstra's algorithm, because it doesn’t need to visit every node of the graph.

Grid world navigation      |  Graph-based representation
:-------------------------:|:-------------------------:
<img src="images/path-finding.jpg" style="width: 400px;">  |  <img src="images/tree_expansion.png" style="width: 400px;">

## A* Pseudocode

<img src="images/Astar.jpg" style="width: 600px;"> 

##### Use and explore graph search algorithm 

Follow the link below to explore graph search algorithm, like Dijkstra and A* 

https://qiao.github.io/PathFinding.js/visual/

### Implementation

In [ ]:
class GraphSearch:

    def __init__(self, ox, oy, reso, rr):
        """
        Initialize map for a star planning
        ox: x position list of Obstacles [m]
        oy: y position list of Obstacles [m]
        reso: grid resolution [m]
        rr: robot radius[m]
        """

        self.reso = reso
        self.rr = rr
        self.calc_obstacle_map(ox, oy)
        self.motion = self.get_motion_model()
    
    # graph representation of the grid     
    class Node:
        def __init__(self, x, y, cost, pind):
            self.x = x  # index of grid
            self.y = y  # index of grid
            self.cost = cost
            self.pind = pind   # parent id

        def __str__(self):
            return str(self.x) + "," + str(self.y) + "," + str(self.cost) + "," + str(self.pind)

    # -------------------------------------------------------
    #  Back-end functions
    # -------------------------------------------------------

    def calc_position(self, index, minp):
        pos = index*self.reso+minp
        return pos

    def calc_xyindex(self, position, minp):
        return round((position - minp)/self.reso)

    def calc_index(self, node):
        return (node.y - self.miny) * self.xwidth + (node.x - self.minx)
    
    # verify that node is collision free     
    def verify_node(self, node):
        px = self.calc_position(node.x, self.minx)
        py = self.calc_position(node.y, self.miny)

        if px < self.minx:
            return False
        elif py < self.miny:
            return False
        elif px >= self.maxx:
            return False
        elif py >= self.maxy:
            return False
    
        # check for collision         
        if self.obmap[node.x][node.y]:
            return False

        return True

    def calc_obstacle_map(self, ox, oy):

        self.minx = round(min(ox))
        self.miny = round(min(oy))
        self.maxx = round(max(ox))
        self.maxy = round(max(oy))
        print("minx:", self.minx)
        print("miny:", self.miny)
        print("maxx:", self.maxx)
        print("maxy:", self.maxy)

        self.xwidth = round((self.maxx - self.minx)/self.reso)
        self.ywidth = round((self.maxy - self.miny)/self.reso)
        print("xwidth:", self.xwidth)
        print("ywidth:", self.ywidth)

        # obstacle map generation
        self.obmap = [[False for i in range(self.ywidth)]
                      for i in range(self.xwidth)]
        for ix in range(self.xwidth):
            x = self.calc_position(ix, self.minx)
            for iy in range(self.ywidth):
                y = self.calc_position(iy, self.miny)
                for iox, ioy in zip(ox, oy):
                    d = math.sqrt((iox - x)**2 + (ioy - y)**2)
                    if d <= self.rr:
                        self.obmap[ix][iy] = True
                        break

    def get_motion_model(self):
        # dx, dy, cost
        motion = [[1, 0, 1],
                  [0, 1, 1],
                  [-1, 0, 1],
                  [0, -1, 1],
                  [-1, -1, math.sqrt(2)],
                  [-1, 1, math.sqrt(2)],
                  [1, -1, math.sqrt(2)],
                  [1, 1, math.sqrt(2)]]

        return motion
    
    
    # -------------------------------------------------------
    #  Main Algorithm
    # -------------------------------------------------------

 
    def planning(self, sx, sy, gx, gy):
        """
        graph path search
        input:
            sx: start x position [m]
            sy: start y position [m]
            gx: goal x position [m]
            gx: goal x position [m]
        output:
            rx: x position list of the final path
            ry: y position list of the final path
        """

        nstart = self.Node(self.calc_xyindex(sx, self.minx),
                           self.calc_xyindex(sy, self.miny), 0.0, -1)
        ngoal = self.Node(self.calc_xyindex(gx, self.minx),
                          self.calc_xyindex(gy, self.miny), 0.0, -1)

        openset, closedset = dict(), dict()
        openset[self.calc_index(nstart)] = nstart
    
        expansion = []
        while 1:
            c_id = min(openset, key=lambda o: openset[o].cost)
            #c_id = min(openset, key=lambda o: openset[o].cost + self.calc_heuristic(ngoal, openset[o]))
            current = openset[c_id]
            
            # store expansion for visualisation reasons             
            expansion.append([self.calc_position(current.x, self.minx),
                              self.calc_position(current.y, self.miny)])
                
            #  if goal was found, stop !         
            if current.x == ngoal.x and current.y == ngoal.y:
                print("Found goal")
                ngoal.pind = current.pind
                ngoal.cost = current.cost
                break

            # Remove the item from the open set
            del openset[c_id]

            # Add it to the closed set
            closedset[c_id] = current

            # expand search grid based on motion model, based on all the actions
            for i, _ in enumerate(self.motion):
                node = self.Node(current.x + self.motion[i][0],
                                 current.y + self.motion[i][1],
                                 current.cost + self.motion[i][2], c_id)
                n_id = self.calc_index(node)

                if n_id in closedset:
                    continue
                
                if not self.verify_node(node):
                    continue
                
                # if a new node is discovered
                if n_id not in openset:
                    openset[n_id] = node  
                else:
                    # if the node is not new, but the new path to the node is better then 
                    # still add the node to the openset                    
                    if openset[n_id].cost > node.cost:
                        # This path is the best until now. record it!
                        openset[n_id] = node

        # find the optimal path to the goal         
        rx, ry = self.calc_final_path(ngoal, closedset)

        return rx, ry, expansion

    def calc_final_path(self, ngoal, closedset):
        # generate final course
        rx, ry = [self.calc_position(ngoal.x, self.minx)], [
                  self.calc_position(ngoal.y, self.miny)]
        pind = ngoal.pind
        while pind != -1:
            n = closedset[pind]
            rx.append(self.calc_position(n.x, self.minx))
            ry.append(self.calc_position(n.y, self.miny))
            pind = n.pind

        return rx, ry
    
    def calc_heuristic(self, n1, n2):
        w = 1.0  # weight of heuristic
        d = w * math.sqrt((n1.x - n2.x)**2 + (n1.y - n2.y)**2)
        return d

### Create a world 

In [ ]:
# start and goal position
sx = -5.0  # [m]
sy = -5.0  # [m]
gx = 50.0  # [m]
gy = 50.0  # [m]
grid_size = 2.0  # [m]
robot_radius = 1.0  # [m]

# set obstacle positions
ox, oy = [], []
for i in range(-10, 60):
    ox.append(i)
    oy.append(-10.0)
for i in range(-10, 60):
    ox.append(60.0)
    oy.append(i)
for i in range(-10, 61):
    ox.append(i)
    oy.append(60.0)
for i in range(-10, 61):
    ox.append(-10.0)
    oy.append(i)
for i in range(-10, 40):
    ox.append(20.0)
    oy.append(i)
for i in range(0, 40):
    ox.append(40.0)
    oy.append(60.0 - i)

### Use the GraphSearch class to find the path to the goal 

* Plot samples from graph expansion phase
* Plot solution path

In [ ]:
# --------------------------------------------------
# Instanciate the graph search class
# and search for a path to the goal
# --------------------------------------------------
graphSearch = GraphSearch(ox, oy, grid_size, robot_radius)
rx, ry, expansion = graphSearch.planning(sx, sy, gx, gy)

# visualise the graph expansion phase
expansion_np = np.array(expansion)
for i in range(0,len(expansion), 100):
    plt.figure(i)
    plt.plot(ox, oy, ".k")
    plt.plot(sx, sy, "og")
    plt.plot(gx, gy, "xb")
    plt.grid(True)
    plt.axis("equal")
    plt.plot(expansion_np[:i,0], expansion_np[:i,1], "xc")
    plt.show()

# visualise the solution path     
plt.figure(2)
plt.plot(ox, oy, ".k")
plt.plot(sx, sy, "og")
plt.plot(gx, gy, "xb")
plt.grid(True)
plt.axis("equal")
plt.plot(rx, ry, "-r")
plt.show()

**References:**

Atsushi Sakai (Modified by Jack Wilkinson and Theodoros Stouraitis)

https://github.com/AtsushiSakai/PythonRobotics/tree/master/PathPlanning

https://pythonrobotics.readthedocs.io/en/latest/modules/path_planning.html#grid-based-search    


In [ ]:
%reset

# Part B: Rapidly-Exploring Random Trees (RRT)

## Import Packages

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Outline of Part B
In this part of the tutorial, we will investigate a sampling-based planning algorithm called RRT. Below are some of the learning objectives you should achieve by the end of the tutorial.

**Objectives:**
* Understand the advantages of RRT compared to Graph Search algorithms
* Gain an understanding of how RRT works
* Implement the RRT path planning algorithm

## RRT Algorithm
Rapidly-Exploring Randoms Trees (RRT) is a sampling-based planning algorithm which makes use of probabilistic methods and a roadmap. The roadmap shown below is created by RRT and represents a network of potential obstacle free paths through the environment. The main advantage of RRT in comparison to algorithms such as A* is the use of random sampling which greatly reduces computational cost when analysing the map. A limitation of this approach is that computed paths are non-optimal, due to the random sampling of nodes.

The graph or roadmap below is an abstract representation of a set of objects or nodes/vertices connected by links or edges. The edges can be directed, in this case the starting point is $q_1$. A sequence of connected edges is a path. 

## RRT Pseudocode

<img src="images/rrt_qnear.png" style="width: 400px;">

<img src="images/rrt_pseudo.png" style="width: 900px;">

Inputs: 
* Initial configuration/position $q_{init}$
* Number of nodes, K. In our case we use a while loop until one of the nodes is within a certain radius of the goal.
* Incremental distance or step size $\Delta q$

Looping until target is reached:
1. A random configuration $q_{rand}$ is chosen. $q_{rand}$ is discarded if it lies within an obstacle.
2. The closest node/vertex $q_{near}$ is found. Near being defined in terms of a cost function that includes distance and orientation. $q_{near}$ will not be added to the graph if the path from $q_{near}$ to $q_{rand}$ intersects an obstacle. 
3. The point $q_{new}$ is added, which is the distance the robot can move in a fixed path towards $q_{rand}$ from $q_{near}$.

## Voronoi Bias
In the Voronoi diagram below we can see each cell or region corresponds to a site within the cell and consists of all points that are closer to its site than to any other site. The edges of the cells are the points that are equidistant to the two nearest sites. 

The key idea in sampling-based planners is using Voronoi bias in the exploration process by expanding the node in the tree that is closest ($q_{near}$) to the random point ($q_{rand}$) in each iteration. By using random samples, the probability that a vertex/node is chosen is proportional to the volume of its Voronoi region. This encourages tree growth to the larger unexplored regions. 

<img src="images/voronoi.png" style="width: 400px;">

<img src="images/rrt_tree.gif" style="width: 600px;">

### Define RRT Class

In [ ]:
#Global configuration
RADIUS_OBSTACLE = 0.8
RADIUS_TARGET = 1.0
RRT_EXTEND_DIST = 1.0
SMOOTHING_ITERATIONS = 200
SMOOTHING_STEP = 0.1

#Global starting position (x, y)
start = (4.0, -5.0)
#Global target position (x, y)
target = (-6.0, 8.0)


class RRT():
    
    def __init__(self):
        
        # circle obstacles list [(x, y)]
        self.obstacles = []
        
        # tree nodes list [((pos_x, pos_y), parent_index)]
        self.nodes = [(start, None)]
        
    #Add a line of obstacles between the two given points
    def generate_obstacles_line(self, q1, q2):
        length = np.sqrt(np.power(q1[0]-q2[0], 2) + np.power(q1[1]-q2[1], 2))
        vectx = (q2[0]-q1[0])/length
        vecty = (q2[1]-q1[1])/length
        for l in np.arange(0.0, length, 1.9*RADIUS_OBSTACLE):
            self.obstacles += [(q1[0]+l*vectx, q1[1]+l*vecty)]

    #Return True if the given point is colliding 
    def check_point_collision(self, q):
        for o in self.obstacles:
            dist = np.sqrt(np.power(q[0]-o[0], 2) + np.power(q[1]-o[1], 2))
            if dist <= RADIUS_OBSTACLE:
                return True
        return False

    #Return True if the segment between the two 
    #given points is colliding the obstacles
    def check_segment_collision(self, q1, q2):
        length = np.sqrt(np.power(q1[0]-q2[0], 2) + np.power(q1[1]-q2[1], 2))
        vectx = (q2[0]-q1[0])/length
        vecty = (q2[1]-q1[1])/length
        for l in np.arange(0.0, length, 0.6*RADIUS_OBSTACLE):
            q = (q1[0]+l*vectx, q1[1]+l*vecty)
            if self.check_point_collision(q):
                return True
        return False

    #Return a random point in the working space
    def sample_point(self):
        x = random.uniform(-10.0, 10.0)
        y = random.uniform(-10.0, 10.0)
        return (x,y)

    #Return the index of the closest node in RRT from given point
    def find_nearest_neighbour(self, q):
        index = 0
        dist_min = float('inf')
        for (i, n) in enumerate(self.nodes):
            dist = np.sqrt(np.power(q[0]-n[0][0], 2) + np.power(q[1]-n[0][1], 2))
            if dist < dist_min:
                dist_min = dist;
                index = i
        return index
    
    def find_new_reachable_pt(self, q_sample, q_near):
        
        dist = np.sqrt(np.power(q_sample[0]- q_near[0], 2) + np.power(q_sample[1]- q_near[1], 2))
        if dist > RRT_EXTEND_DIST:
            q_normalized_x = q_near[0] + RRT_EXTEND_DIST*(q_sample[0]- q_near[0])/dist
            q_normalized_y = q_near[1] + RRT_EXTEND_DIST*(q_sample[1]- q_near[1])/dist
        else:
            q_normalized_x = q_sample[0]
            q_normalized_y = q_sample[1]
        q_new = (q_normalized_x, q_normalized_y)

        return q_new
    

### Implement RRT Algorithm

In [ ]:
rrt_plan = RRT()

# Define obstacle lines
rrt_plan.generate_obstacles_line((-12.0, 4.0), (5.0, 5.0))
rrt_plan.generate_obstacles_line((-2.0, -1.0), (12.0, -3.0))


#Sample and grow the RRT until target is reached
is_reached = False
while not is_reached:
    
    # Sample a new point  
    ### START CODE HERE (~1 line of code)

    ### END CODE HERE
    
    # Check that the new point is collision free
    if not rrt_plan.check_point_collision(q_sample):

        # Find the index of the nearest node (q_near) in the graph    
        ### START CODE HERE (~1 line of code)

        ### END CODE HERE
        
        q_near = rrt_plan.nodes[index][0]

        # Find the closest feasible point to the randomly sampled point (q_new)
        ### START CODE HERE (~1 line of code)

        ### END CODE HERE
        
        # Check if the edge is collision free
        ### START CODE HERE (~1 line of code)

        ### END CODE HERE
        
            rrt_plan.nodes += [(q_new, index)]

            # Check if the goal has been reached
            if np.sqrt(np.power(q_new[0]-target[0], 2) + np.power(q_new[1]-target[1], 2)) < RADIUS_TARGET:
                is_reached = True

#Retrieve computed path from the tree
#(in reversed order)
path = []
index = len(rrt_plan.nodes)-1
path += [target]
while index != None:
    path += [rrt_plan.nodes[index][0]]
    index = rrt_plan.nodes[index][1]
path += [start]

#Path smoothing
for k in range(SMOOTHING_ITERATIONS):
    index1 = random.randint(0, len(path)-1)
    index2 = random.randint(0, len(path)-1)
    if index1 != index2 and not rrt_plan.check_segment_collision(path[index1], path[index2]):
        if index1 < index2:
            index_low = index1
            index_up = index2
        else:
            index_low = index2
            index_up = index1
        middle = []
        deltax = (path[index_up][0]-path[index_low][0])
        deltay = (path[index_up][1]-path[index_low][1])
        for l in np.arange(SMOOTHING_STEP, 1.0-SMOOTHING_STEP, SMOOTHING_STEP):
            middle += [(path[index_low][0]+l*deltax, path[index_low][1]+l*deltay)]
        path = path[:index_low+1] + middle + path[index_up:]

        

### Plotting the RRT Roadmap

In [ ]:
%matplotlib inline

node_prog = np.linspace(7, len(rrt_plan.nodes), 10, int)

# Plot growing tree
for j in range(len(node_prog)): 
    
    fig = plt.subplots()
    
    #Draw obstacles
    for o in rrt_plan.obstacles:
        circle = plt.Circle(o, RADIUS_OBSTACLE, color='b', fill=False)
        plt.gcf().gca().add_artist(circle)

    #Draw start and target points
    circle_start_1 = plt.Circle(start, RADIUS_TARGET, color='g', alpha=0.5)
    circle_start_2 = plt.Circle(start, RADIUS_OBSTACLE, color='g')
    circle_target_1 = plt.Circle(target, RADIUS_TARGET, color='r', alpha=0.5)
    circle_target_2 = plt.Circle(target, RADIUS_OBSTACLE, color='r')
    plt.gcf().gca().add_artist(circle_start_1)
    plt.gcf().gca().add_artist(circle_start_2)
    plt.gcf().gca().add_artist(circle_target_1)
    plt.gcf().gca().add_artist(circle_target_2)

    
    #Draw tree
    for n in rrt_plan.nodes[0:int(node_prog[j])]:
        if (n[1] != None):
            x1 = n[0][0]
            y1 = n[0][1]
            parent = n[1]
            x2 = rrt_plan.nodes[parent][0][0]
            y2 = rrt_plan.nodes[parent][0][1]
            plt.plot([x1, x2], [y1, y2], color='y', marker='.')
    
    plt.axis('scaled')
    plt.grid()
    plt.gcf().gca().set_xlim((-10.0, 10.0))
    plt.gcf().gca().set_ylim((-10.0, 10.0))
    
    # Final plot
    if(j >= len(node_prog)-1):
    
        #Draw Path
        for i in range(len(path)):
            if i > 0:
                plt.plot([path[i-1][0], path[i][0]], [path[i-1][1], path[i][1]], color='r', marker='.')
        
    plt.show()

**References**

Corke, P., 2017. Robotics, vision and control: fundamental algorithms in MATLAB® second, completely revised (Vol. 118). Springer.
https://en.wikipedia.org/wiki/Rapidly-exploring_random_tree
